In [1]:
import oars.pep
from oars.matrices import *
import numpy as np
np.set_printoptions(precision=3, suppress=True, linewidth=200)
from importlib import reload
import algorithm_test


Testing quadratic
Converged in objective value, iteration 69
Testing L1
Converged in objective value, iteration 16
Testing quadratic
Starting Parallel Algorithm


In [1]:
reload(algorithm_test)

NameError: name 'reload' is not defined

In [2]:
algorithm_test.testSDP(parallel=True)


In [54]:
from oars.matrices import getFull, getMT
from oars.pep import getConstraintMatrices
import proxs
reload(proxs)
from oars import solveMT
n = 3
Z, W = getMT(n)
Ko, K1, Ki, Kp = getConstraintMatrices(Z, W, gamma=0.5)
u = np.ones(2*n)
u[n:] = 0
# proxlist = [proxs.traceHalfspaceIndicator, proxs.traceEqualityIndicator, proxs.traceHalfspaceIndicator,]# for _ in Kp]
# proxlist = []
proxlist = [proxs.psdCone, proxs.traceEqualityIndicator, proxs.traceEqualityIndicator, proxs.linearSubdiff] + [proxs.traceHalfspaceIndicator for _ in Kp] #, linearSubdiff]
data = [(2*n, 2*n), {'A':Ki, 'v':1}, {'A':K1, 'v':0}, -Ko] + Kp #, u, {'A':Ki, 'v':1}]#, -Ko]
#data = [Kp[0], {'A':Ki, 'v':1}, Kp[1]]# + [u, (2*n, 2*n)]#, -Ko]
dim = len(data)
Wd, Zd = getBlockMin(dim, dim//2)
from oars import solve
x, results = solve(dim, data, proxlist, W=Wd, Z=Zd, itrs=100000, vartol=1e-4, gamma=0.8, checkperiod=10, verbose=False)
# x, results = solveMT(dim, data, proxlist, itrs=10000, vartol=1e-6, gamma=0.8, checkperiod=10, verbose=False)
oneprox = proxs.traceEqualityIndicator({'A':Ki, 'v':1})
print(oneprox.prox(x))
print(x)
# print(x)
# print(results)
# print(np.trace(Ko @ x))


Converged in objective value, iteration 35356
[[ 0.502 -0.022 -0.474  0.143  0.095 -0.053]
 [-0.022  0.029 -0.004  0.023  0.005  0.02 ]
 [-0.474 -0.004  0.469 -0.161 -0.098  0.034]
 [ 0.143  0.023 -0.161  0.072  0.037  0.003]
 [ 0.095  0.005 -0.098  0.037  0.021 -0.004]
 [-0.053  0.02   0.034  0.003 -0.004  0.017]]
[[ 0.502 -0.022 -0.474  0.143  0.095 -0.053]
 [-0.022  0.029 -0.004  0.023  0.005  0.02 ]
 [-0.474 -0.004  0.469 -0.161 -0.098  0.034]
 [ 0.143  0.023 -0.161  0.072  0.037  0.003]
 [ 0.095  0.005 -0.098  0.037  0.021 -0.004]
 [-0.053  0.02   0.034  0.003 -0.004  0.017]]


In [55]:
np.trace(Ko @ x)

0.8806363180682683

In [53]:
from oars.pep import getContractionFactor
print(getContractionFactor(Z, W, verbose=True))

optimal
0.8772093960780886
[[ 0.497 -0.024 -0.473  0.142  0.092 -0.056]
 [-0.024  0.027 -0.003  0.021  0.004  0.019]
 [-0.473 -0.003  0.476 -0.163 -0.097  0.037]
 [ 0.142  0.021 -0.163  0.071  0.036  0.002]
 [ 0.092  0.004 -0.097  0.036  0.02  -0.005]
 [-0.056  0.019  0.037  0.002 -0.005  0.017]]
1.057491839126227
0.24253000172637015
1.8391259270014304
0.2831436413187557
2.0488014245113737
0.365486443518793
0.877208753564782
804.0745487528972
0.8772093960780886


In [49]:
for Kpi in Kp:
    print(np.trace(Kpi @ x))
    half = proxs.traceHalfspaceIndicator(Kpi)
    y = half.prox(x)
    print(np.trace(Kpi @ y))
    print(np.linalg.eigvalsh(Kpi))

cone = proxs.reducedCone(u)
yy = cone.prox(x)
print(yy)
print(np.linalg.eigvalsh(yy))
cone.getEigen(x)
print(np.linalg.eigvalsh(x))
# print(cone.prox(x))

-1.726780265748351e-05
0.0
[-2.118  0.     0.    -0.    -0.     0.118]
-5.286842805712899e-05
-5.551115123125783e-17
[-1.236 -0.     0.     0.     0.     3.236]
-1.3989199108756234e-05
0.0
[-2.225 -0.     0.     0.    -0.     0.225]
3.3899369720605255e-05
3.3899369720605255e-05
[-2.372 -0.    -0.     0.     0.     3.372]
-0.00018755979739369655
3.469446951953614e-18
[-2.323 -0.     0.     0.     0.     0.323]
-0.0003738813230230731
1.3877787807814457e-17
[-3.464 -0.    -0.    -0.     0.     3.464]
[[ 0.508 -0.021 -0.478  0.146  0.101 -0.044]
 [-0.021  0.03  -0.012  0.028  0.007  0.022]
 [-0.478 -0.012  0.451 -0.158 -0.101  0.027]
 [ 0.146  0.028 -0.158  0.073  0.039  0.007]
 [ 0.101  0.007 -0.101  0.039  0.023 -0.002]
 [-0.044  0.022  0.027  0.007 -0.002  0.016]]
[-0.012 -0.     0.     0.     0.081  1.032]
[-0.001 -0.    -0.     0.     0.082  1.032]


In [25]:
# [[ 0.786  0.521  0.521  0.142  0.     0.   ]
#  [ 0.521  0.857  0.502 -0.041  0.     0.   ]
#  [ 0.521  0.502  0.857 -0.041  0.     0.   ]
#  [ 0.142 -0.041 -0.041  0.08   0.     0.   ]
#  [ 0.     0.     0.     0.     0.     0.   ]
#  [ 0.     0.     0.     0.     0.     0.   ]]

xx = np.array([[ 0.786,  0.521,  0.521,  0.142,  0.   ,  0.   ],
    [ 0.521,  0.857,  0.502, -0.041,  0.   ,  0.   ],
        [ 0.521,  0.502,  0.857, -0.041,  0.   ,  0.   ],
        [ 0.142, -0.041, -0.041,  0.08 ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

np.linalg.eig(xx)

EigResult(eigenvalues=array([ 1.864,  0.362, -0.   ,  0.355,  0.   ,  0.   ]), eigenvectors=array([[-0.566, -0.723, -0.396, -0.   ,  0.   ,  0.   ],
       [-0.583,  0.358,  0.178, -0.707,  0.   ,  0.   ],
       [-0.583,  0.358,  0.178,  0.707,  0.   ,  0.   ],
       [-0.018, -0.469,  0.883, -0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ]]))

In [21]:
traceeq = traceEqualityIndicator({'A':Ki, 'v':1})
print(traceeq.U)

[[0.333 0.    0.    0.    0.    0.   ]
 [0.    0.333 0.    0.    0.    0.   ]
 [0.    0.    0.333 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.   ]]


In [15]:
W@W

array([[ 1.  ,  0.  ,  0.  , -1.  ,  0.5 ,  0.5 ],
       [ 0.  ,  1.  ,  0.  ,  0.5 , -1.  ,  0.5 ],
       [ 0.  ,  0.  ,  1.  ,  0.5 ,  0.5 , -1.  ],
       [-1.  ,  0.5 ,  0.5 ,  1.5 , -0.75, -0.75],
       [ 0.5 , -1.  ,  0.5 , -0.75,  1.5 , -0.75],
       [ 0.5 ,  0.5 , -1.  , -0.75, -0.75,  1.5 ]])

In [16]:
W@W

array([[ 6., -3., -3.],
       [-3.,  6., -3.],
       [-3., -3.,  6.]])

In [19]:
y = oneprox.prox(x)
np.trace(Ki @ y)
Ki

array([[1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [9]:
import proxs
import timeit
X = np.random.randn(10, 10)

# Make symmetric A
A = X + X.T

aprox = proxs.traceHalfspaceIndicator(A)
A.shape[0]
# print(aprox.scale(A))
# print(aprox.other_scale(A))
# np.allclose(aprox.scale(A), aprox.other_scale(A))
# # Compare the times for scale and otherscale
print(timeit.timeit('aprox.scale(A)', number=1000, globals=globals()))

# timeit.timeit('aprox.other_scale(A)', number=1000, globals=globals())
# q = 0
# for i in range(A.shape[0]):
#     for j in range(i):
#         q += 2*A[i,j]**2
#     q += A[i,i]**2

# print(q)
# print(np.linalg.norm(A, 'fro')**2)

0.006233800027985126


In [13]:
reload(proxs)

<module 'proxs' from 'c:\\Users\\peter\\OneDrive\\Documents\\code\\oars\\tests\\proxs.py'>

In [13]:
Y = 2*A + np.eye(A.shape[0])
np.trace(A@Y)

350.94063175498553

In [14]:
aprox = proxs.traceEqualityIndicator({'A':A, 'v':1})
Yp = aprox.prox(A, 1)
print(np.trace(A@Yp))

1.0000000000000016


In [5]:
np.linalg.norm(A, 'fro')**2

255.65986269415086

In [2]:
n=4
Z, W = getMT(n)
L = - np.tril(Z, -1)
ls = np.ones(n)*2
ls = np.cumsum(ls)
mus = np.ones(n)
mus = np.cumsum(mus)
operators = [oars.pep.SmoothStronglyConvexFunction(L=ls[i], mu=mus[i]) for i in range(n)]
tau = oars.pep.getContractionFactor(Z, W, None, None, operators=operators, alpha=1, gamma=0.5)
W, tau_opt = oars.pep.getOptimalW(Z, operators=operators)
Wo, tauo = oars.pep.getSubdiffW(L, ls=ls, mus=mus)
print(W)
print(tau_opt)
print(tau)
print(Wo)
print(tauo)

[[ 2.43588727 -1.09730184 -0.25396727 -1.08461816]
 [-1.09730184  3.237486   -1.4169987  -0.72318545]
 [-0.25396727 -1.4169987   3.79612127 -2.1251553 ]
 [-1.08461816 -0.72318545 -2.1251553   3.93295892]]
0.14316698043646195
0.8993338570188733
[[ 2.43588727 -1.09730184 -0.25396727 -1.08461816]
 [-1.09730184  3.237486   -1.4169987  -0.72318545]
 [-0.25396727 -1.4169987   3.79612127 -2.1251553 ]
 [-1.08461816 -0.72318545 -2.1251553   3.93295892]]
0.14316698043646195


In [2]:
n=4
Z, W = getMT(n)
L = - np.tril(Z, -1)
ls = np.ones(n)*2
ls = np.cumsum(ls)
mus = np.ones(n)
mus = np.cumsum(mus)
operators = [oars.pep.LipschitzStronglyMonotoneOperator(L=ls[i], mu=mus[i]) for i in range(n)]
tau = oars.pep.getContractionFactor(Z, W, None, None, operators=operators, alpha=1, gamma=0.5)
W, tau_opt = oars.pep.getOptimalW(Z, operators=operators)
Wo, tauo = oars.pep.getSubdiffW(L, ls=ls, mus=mus)
print(W)
print(tau_opt)
print(tau)
print(Wo)
print(tauo)

[[ 2.10883221 -1.011507   -0.3138347  -0.7834905 ]
 [-1.011507    2.97677056 -1.18071865 -0.7845449 ]
 [-0.3138347  -1.18071865  3.00502589 -1.51047254]
 [-0.7834905  -0.7845449  -1.51047254  3.07850794]]
0.6562344018316276
0.9924137484022097
[[ 2.43588727 -1.09730184 -0.25396727 -1.08461816]
 [-1.09730184  3.237486   -1.4169987  -0.72318545]
 [-0.25396727 -1.4169987   3.79612127 -2.1251553 ]
 [-1.08461816 -0.72318545 -2.1251553   3.93295892]]
0.14316698043646195


In [2]:
n = 4
Z, W = getMT(n)
L = -np.tril(Z, -1)
ls = np.ones(n)*2
mus = np.ones(n)
Wopt, tau = oars.pep.getOptimalW(Z, ls=ls, mus=mus)
print(Wopt)
print(tau)

[[ 6.98255568e-09 -6.92177773e-09  3.66397885e-11  3.66397881e-11]
 [-6.92177773e-09  7.39331880e-09 -1.95265932e-10 -1.95265931e-10]
 [ 3.66397885e-11 -1.95265932e-10  1.85725841e-11  1.85725869e-11]
 [ 3.66397881e-11 -1.95265931e-10  1.85725869e-11  1.85725855e-11]]
1.0000000006201917


In [4]:
reload(oars.pep)

<module 'oars.pep' from 'C:\\Users\\peter\\OneDrive\\Documents\\code\\oars\\oars\\pep\\__init__.py'>

In [4]:
n = 3
Z, W = getMT(n)
M = getIncidence(W)

Ko, K1, Ki, Kp = getConstraintMatrices(Z, W)
Ko

array([[ 1.,  0.,  0., -1.,  1., -0.],
       [ 0.,  1.,  0.,  1., -2.,  1.],
       [ 0.,  0.,  1., -0.,  1., -1.],
       [-1.,  1., -0.,  2., -3.,  1.],
       [ 1., -2.,  1., -3.,  6., -3.],
       [-0.,  1., -1.,  1., -3.,  2.]])

In [5]:
K1

array([[1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [6]:
Ki

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [8]:
n = 3
Z, W = getMT(n)
eye = np.eye(n)
ones = np.ones((n,n))
zeros = np.zeros((n,n))
Ko, K1, Ki, Kp = getConstraintMatrices(Z, W)
Ko_ref = np.block([[eye, -W], [-W, W@W]])
assert(np.allclose(Ko, Ko_ref))

In [10]:
K1_ref = np.block([[ones, zeros], [zeros, zeros]])
assert(np.allclose(K1, K1_ref))
Ki_ref = np.block([[eye, zeros], [zeros, zeros]])
assert(np.allclose(Ki, Ki_ref))

In [13]:
for k in Kp:
    print(str(k))

[[-1. -0. -0.  1.  0.  0.]
 [-0. -0. -0.  0.  0.  0.]
 [-0. -0. -0.  0.  0.  0.]
 [ 1.  0.  0.  3.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.   0.   0.   0.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.5  0.   0.  -2.  -0.  -0. ]
 [ 0.   0.   0.  -0.  -0.  -0. ]
 [ 0.   0.   0.  -0.  -0.  -0. ]]
[[-0. -0. -0. -0.  0.  0.]
 [-0. -1. -0. -1.  1.  0.]
 [-0. -0. -0. -0.  0.  0.]
 [-0. -1. -0. -1.  1.  0.]
 [ 0.  1.  0.  1.  3.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
[[ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.5  0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.5  0. ]
 [ 0.   0.5  0.   0.5 -2.  -0. ]
 [ 0.   0.   0.   0.  -0.  -0. ]]
[[-0. -0. -0. -0. -0.  0.]
 [-0. -0. -0. -0. -0.  0.]
 [-0. -0. -1. -1. -1.  1.]
 [-0. -0. -1. -1. -1.  1.]
 [-0. -0. -1. -1. -1.  1.]
 [ 0.  0.  1.  1.  1.  3.]]
[[ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.5]
 [ 0.   0.   0

In [14]:
Kp

[array([[-1., -0., -0.,  1.,  0.,  0.],
        [-0., -0., -0.,  0.,  0.,  0.],
        [-0., -0., -0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  3.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. ,  0. ,  0.5,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0.5,  0. ,  0. , -2. , -0. , -0. ],
        [ 0. ,  0. ,  0. , -0. , -0. , -0. ],
        [ 0. ,  0. ,  0. , -0. , -0. , -0. ]]),
 array([[-0., -0., -0., -0.,  0.,  0.],
        [-0., -1., -0., -1.,  1.,  0.],
        [-0., -0., -0., -0.,  0.,  0.],
        [-0., -1., -0., -1.,  1.,  0.],
        [ 0.,  1.,  0.,  1.,  3.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ],
        [ 0. ,  0.5,  0. ,  0.5, -2. , -0. ],
        

In [15]:
n = 3
Z, W = getMT(n)
M = getIncidence(W)
Ko, Ki, Kmus, Kls = getGramsOld(Z, M)
Kon, Kin, Kpn = getReducedConstraintMatrices(Z, M)

No operators provided, using default LipschitzStronglyMonotoneOperator


In [16]:
Kon

array([[ 1.,  0., -1.,  1.,  0.],
       [ 0.,  1.,  0., -1.,  1.],
       [-1.,  0.,  1., -1.,  0.],
       [ 1., -1., -1.,  2., -1.],
       [ 0.,  1.,  0., -1.,  1.]])

In [19]:
n = 3
Z, W = getMT(n)
M = getIncidence(W)
d = M.shape[0]
Ko, Ki, Kmus, Kls = getGramsOld(Z, M)
Kon, Kin, Kpn = getReducedConstraintMatrices(Z, M)
eye = np.eye(d)
Ko_ref = np.block([[eye, M], [M.T, M.T@M]])
assert(np.allclose(Ko_ref, Kon))

No operators provided, using default LipschitzStronglyMonotoneOperator


In [20]:
Ki

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [22]:

    Ki_ref = np.zeros((d+n, d+n))
Ki_ref[:d, :d] = eye
assert(np.allclose(Ki_ref, Kin))

In [23]:
Kpn

[array([[-1.,  0.,  1.,  0.,  0.],
        [ 0., -0., -0., -0., -0.],
        [ 1., -0.,  3.,  0.,  0.],
        [ 0., -0.,  0.,  0.,  0.],
        [ 0., -0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. ,  0.5,  0. ,  0. ],
        [ 0. ,  0. , -0. , -0. , -0. ],
        [ 0.5, -0. , -2. , -0. , -0. ],
        [ 0. , -0. , -0. , -0. , -0. ],
        [ 0. , -0. , -0. , -0. , -0. ]]),
 array([[-1.,  1.,  1., -1., -0.],
        [ 1., -1., -1.,  1.,  0.],
        [ 1., -1., -1.,  1.,  0.],
        [-1.,  1.,  1.,  3.,  0.],
        [-0.,  0.,  0.,  0.,  0.]]),
 array([[ 0. ,  0. , -0. , -0.5, -0. ],
        [ 0. ,  0. ,  0. ,  0.5,  0. ],
        [-0. ,  0. ,  0. ,  0.5,  0. ],
        [-0.5,  0.5,  0.5, -2. , -0. ],
        [-0. ,  0. ,  0. , -0. , -0. ]]),
 array([[-0., -0.,  0.,  0., -0.],
        [-0., -1.,  1.,  1., -1.],
        [ 0.,  1., -1., -1.,  1.],
        [ 0.,  1., -1., -1.,  1.],
        [-0., -1.,  1.,  1.,  3.]]),
 array([[ 0. ,  0. , -0. , -0. , -0. ],
        [ 0. ,  0. , -0. 

In [24]:
Kpn_ref = [np.array([[-1.,  0.,  1.,  0.,  0.],
                [ 0., -0., -0., -0., -0.],
                [ 1., -0.,  3.,  0.,  0.],
                [ 0., -0.,  0.,  0.,  0.],
                [ 0., -0.,  0.,  0.,  0.]]),
        np.array([[ 0. ,  0. ,  0.5,  0. ,  0. ],
                [ 0. ,  0. , -0. , -0. , -0. ],
                [ 0.5, -0. , -2. , -0. , -0. ],
                [ 0. , -0. , -0. , -0. , -0. ],
                [ 0. , -0. , -0. , -0. , -0. ]]),
        np.array([[-1.,  1.,  1., -1., -0.],
                [ 1., -1., -1.,  1.,  0.],
                [ 1., -1., -1.,  1.,  0.],
                [-1.,  1.,  1.,  3.,  0.],
                [-0.,  0.,  0.,  0.,  0.]]),
        np.array([[ 0. ,  0. , -0. , -0.5, -0. ],
                [ 0. ,  0. ,  0. ,  0.5,  0. ],
                [-0. ,  0. ,  0. ,  0.5,  0. ],
                [-0.5,  0.5,  0.5, -2. , -0. ],
                [-0. ,  0. ,  0. , -0. , -0. ]]),
        np.array([[-0., -0.,  0.,  0., -0.],
                [-0., -1.,  1.,  1., -1.],
                [ 0.,  1., -1., -1.,  1.],
                [ 0.,  1., -1., -1.,  1.],
                [-0., -1.,  1.,  1.,  3.]]),
        np.array([[ 0. ,  0. , -0. , -0. , -0. ],
                [ 0. ,  0. , -0. , -0. , -0.5],
                [-0. , -0. ,  0. ,  0. ,  0.5],
                [-0. , -0. ,  0. ,  0. ,  0.5],
                [-0. , -0.5,  0.5,  0.5, -2. ]])]
for i in range(2*n):
    assert(np.allclose(Kpn[i], Kpn_ref[i]))